In [23]:
import numpy as np
from geopandas import points_from_xy
import scipy.sparse as sp

from climada.hazard import Hazard, Centroids
from climada.entity import Exposures, ImpactFuncSet, ImpactFunc
from climada.engine import Impact, ImpactCalc

# One hazard with 10 events
size_hazard = 1900
events = 20
lat, lon = np.meshgrid(
    np.linspace(0.1, 1.1, num=size_hazard), np.linspace(0.1, 1.1, num=size_hazard)
)
lat, lon = lat.flatten(), lon.flatten()
centroids = Centroids(lat=lat, lon=lon)
rng = np.random.default_rng(1)
hazard = Hazard(
    haz_type="A",
    centroids=centroids,
    event_id=np.arange(1, events + 1),
    frequency=np.ones(events, dtype="float"),
    event_name=[""] * events,
    date=np.arange(1, events + 1),
    intensity=sp.csr_matrix(rng.random((events, size * size))),
)

# 10 exposures with same coordinates but different values
size_exposure = 2100
lat, lon = np.meshgrid(
    np.linspace(0, 1, num=size_exposure), np.linspace(0, 1, num=size_exposure)
)
lat, lon = lat.flatten(), lon.flatten()
exposure = Exposures(
    data={
        "value": rng.random(size_exposure * size_exposure, dtype="float"),
        "latitude": lat,
        "longitude": lon,
    }
)
exposure.gdf["impf_A"] = 1
exposure.assign_centroids(hazard)
exposure_map = {event_id: exposure.copy() for event_id in hazard.event_id.flat}
for idx, exp in exposure_map.items():
    exp.gdf["value"] = exp.gdf["value"] + idx

# Some impact function
impf_set = ImpactFuncSet(
    [ImpactFunc.from_step_impf(intensity=(0, 0.5, 1), haz_type="A")]
)

In [24]:
import cProfile

# Single exposure
with cProfile.Profile() as pr:
    impact = ImpactCalc(exposures=exposure, impfset=impf_set, hazard=hazard).impact(
        save_mat=False, assign_centroids=False
    )
    pr.dump_stats("single.prof")

In [25]:
# Multi-exposure
with cProfile.Profile() as pr:
    impacts = [
        ImpactCalc(
            exposures=exposure_map[event_id],
            impfset=impf_set,
            hazard=hazard.select(event_id=[event_id]),
        ).impact(save_mat=False, assign_centroids=False)
        for event_id in hazard.event_id.flat
    ]
    # Fix unimportant attributes so we can concatenate
    for impact in impacts:
        for attr in ("crs", "tot_value", "unit", "frequency_unit"):
            setattr(impact, attr, 0)

    impact = Impact.concat(impacts)
    pr.dump_stats("multi.prof")

2024-04-19 17:17:03,635 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
2024-04-19 17:17:03,650 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
2024-04-19 17:17:03,651 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
2024-04-19 17:17:03,651 - climada.engine.impact - WARNING - The Impact.tot_value attribute is deprecated.Use Exposures.affected_total_value to calculate the affected total exposure value based on a specific hazard intensity threshold
2024-04-19 17:17:03,651 - climada.engine.impact - WARNING - The 